<a href="https://colab.research.google.com/github/Satyake/Open-AI-LLM-Langchain-Projects/blob/main/Text_summarizer_SFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[sentencepiece]
!pip install sacrebleu rouge_score
!pip install py7zr
!pip install --upgrade accelerate
!pip install transformers accelerate
!pip install nltk
!pip install datasets
!pip install transformers[torch]


In [ ]:
!pip install accelerate -U

In [ ]:
!pip install --upgrade tensorflow

In [10]:
import pandas as pd
import torch
import nltk
from tqdm import tqdm
from nltk.tokenize import sent_tokenize
from transformers import pipeline, Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [2]:
from datasets import load_dataset, load_from_disk, load_metric
model="google/pegasus-cnn_dailymail"
tokenizer=AutoTokenizer.from_pretrained(model,return_token_type_ids=False)
#model_pegasus=AutoModelForSeq2SeqLM.from_pretrained(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
#finetuning
dataset=load_dataset('samsum')

In [4]:
def convert_examples_to_features(data_in_batch):
    input_encoding=tokenizer(data_in_batch['dialogue'], max_length=1024, truncation=True)

    target_encoding=tokenizer(data_in_batch['summary'], max_length=128, truncation=True)

    return {
        'input_ids': input_encoding['input_ids'],
        'attention_mask': input_encoding['attention_mask'],
        'labels':target_encoding['input_ids'],
    }

In [5]:
dataset_en=dataset.map(convert_examples_to_features)

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

In [6]:
model=AutoModelForSeq2SeqLM.from_pretrained(model)

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
, from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq
seq2seq_data_collator=DataCollatorForSeq2Seq(tokenizer, model=model)

In [11]:
training_args=TrainingArguments(
    output_dir='pegasus-summary', num_train_epochs=1, warmup_steps=10,
    per_device_train_batch_size=1, per_device_eval_batch_size=1, weight_decay=0.01,logging_steps=10,
    evaluation_strategy='epoch'
)

In [12]:

trainer=Trainer(model=model, args=training_args, tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                train_dataset=dataset_en['test'],
                eval_dataset=dataset_en['validation'])

In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.761400,1.568389


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


TrainOutput(global_step=819, training_loss=1.9090176175947737, metrics={'train_runtime': 447.1605, 'train_samples_per_second': 1.832, 'train_steps_per_second': 1.832, 'total_flos': 314017624350720.0, 'train_loss': 1.9090176175947737, 'epoch': 1.0})

In [ ]:
model.save_pretrained('pegasus-samsum-pretrained')

In [12]:
tokenizer.save_pretrained('pegsus-tokenizer')

('pegsus-tokenizer/tokenizer_config.json',
 'pegsus-tokenizer/special_tokens_map.json',
 'pegsus-tokenizer/spiece.model',
 'pegsus-tokenizer/added_tokens.json',
 'pegsus-tokenizer/tokenizer.json')

In [16]:
!zip -r /content/pegasus-samsum-pretrained


zip error: Nothing to do! (/content/pegasus-samsum-pretrained.zip)


In [53]:
tokenizer=AutoTokenizer.from_pretrained('/content/pegsus-tokenizer')

In [ ]:
tokenizer

In [50]:
model=AutoModelForSeq2SeqLM.from_pretrained('/content/pegasus-samsum-pretrained')

In [ ]:
model

In [15]:
sample_text=dataset['train']['dialogue'][1]
sample_text

'Olivia: Who are you voting for in this election? \r\nOliver: Liberals as always.\r\nOlivia: Me too!!\r\nOliver: Great'

In [14]:
from transformers import pipeline


In [25]:
pipeline_object=pipeline('summarization', model=model, tokenizer=tokenizer)


In [27]:
gen_kwargs={'length_penalty': 0.8,'num_beams':8,'max_length': 128}

In [28]:
pipeline_object(sample_text,**gen_kwargs)

Your max_length is set to 128, but your input_length is only 26. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)


[{'summary_text': "Olivia is voting for liberals in this election. Oliver is also voting for liberals, but he doesn't know which one he's voting for yet."}]

In [45]:
sample_text

'Olivia: Who are you voting for in this election? \r\nOliver: Liberals as always.\r\nOlivia: Me too!!\r\nOliver: Great'

In [29]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]


def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"],
                         attention_mask=inputs["attention_mask"],
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score


In [30]:
rouge_names=['rouge1','rouge2']
rouge_metrics=load_metric('rouge')

<ipython-input-30-5993661d1d30>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metrics=load_metric('rouge')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [32]:
score=calculate_metric_on_test_ds(
    dataset['test'][0:10],rouge_metrics,trainer.model, tokenizer, batch_size=2,
    column_text='dialogue',column_summary='summary'
)

100%|██████████| 5/5 [07:36<00:00, 91.25s/it]


In [33]:
rouge_dict=dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=[f'pegasus'])

,rouge1,rouge2
pegasus,0.02286,0.0
